In [ ]:
pip install datasets

#  데이터 불러오고 전처리

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

In [ ]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [ ]:
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/21018 [00:00<?, ? examples/s]

# Trainer API 쓰지 않고 fine-tuning

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
pip install sacrebleu

In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# 1. Dataloader, 배치 만들기
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    collate_fn=data_collator,
    batch_size=8
)

In [ ]:
# 2. optimizer
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
pip install accelerate

- 장치 관리: 모델, 데이터, 옵티마이저 등을 적절한 계산 장치(CPU, GPU, TPU)에 배치합니다.
- 분산 학습: 여러 GPU나 TPU에서 모델 학습을 수행할 수 있도록 지원합니다. 이를 통해 학습 속도를 크게 향상시킬 수 있습니다.

즉, 장치 배치(device placement)를 자동으로 처리

In [ ]:
# 3. Accelerator 분산 학습

from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

In [ ]:
# 4. 학습률 스케쥴러 설정 (optimizer.step()시 마다)

from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch # 총 스텝

lr_scheduler = get_scheduler(
    "linear", # 선형 감소 ...0
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
    """
    learning rate decay가 진행되기 이전, 학습 초기에 learning rate를 서서히 올리는 기법

    Training이 시작될 때, 모든 parameters들은 보통 random values(initialized)이므로,
    최종 solution에서 멀리 떨어져 있다.
    이 때, 너무 큰 learning rate를 사용하면 numerical instability가 발생할 수 있기에,
    초기에 작은 learning rate를 사용하고, training과정이 안정되면 초기 learning rate로 전환하는 방법
    """
)

In [ ]:
# 5. 후처리 (메트릭 쓰지 않는 것 빼곤 거의 동일)
# 평가 부분을 단순화하기 위해 예측과 레이블을 가져와 메트릭 객체가 예상하는 문자열 list로 변환

def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy() # to cpu & tensor to numpy
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

loss.backward()를 호출하면 각 파라미터들의 .grad 값에 변화도가 저장이 된다.

그니까 update된 weight 가지고 새로운 batch에 대해 forward 하고 다시 back하면 새로운 미분 값이 나오는데
초기화 안되면 누적돼서 이상함

어텐션 마스크는 연산이 필요없는 패딩 토큰에 대해 어텐션을 하지 않도록 마스킹해주는 역할
노이즈 제거

In [ ]:
output_dir = "marian-finetuned-kde4-en-to-fr-accelerate"

In [ ]:
# 6. 학습 루프

from tqdm.auto import tqdm # process bar
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # 학습
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)    # forward
        loss = outputs.loss         # loss
        accelerator.backward(loss)  # backward (loss.backward())

        optimizer.step()            # update
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # 평가
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
          # generate()는 Accelerate가 prepare()에서 생성한 것이 아님. 모델 래핑 풀고 generate(번역)
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # 예측과 레이블을 모으기 전에 함께 패딩 수행=========================================
        # 여러 프로세스에서 생성된 토큰을 패딩하여 각 배치가 동일한 길이를 갖도록
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        # 분산 -> 모으기
        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        # 후처리해서 메트릭에 추가
        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # 저장 및 업로드
    accelerator.wait_for_everyone() # to make sure all processes join
    unwrapped_model = accelerator.unwrap_model(model) # save_pretrained() 사용 위해
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process: # once only
        tokenizer.save_pretrained(output_dir)
        #repo.push_to_hub(
        #    commit_message=f"Training in progress epoch {epoch}", blocking=False
        #)

분산 처리 환경에서는 여러 프로세스가 동시에 데이터를 처리하므로, 각 프로세스가 생성한 토큰의 길이는 서로 다를 수 있습니다. 따라서 이러한 토큰들을 하나의 텐서로 모으기 위해서는 길이를 맞춰주는 패딩 과정이 필요합니다.

https://huggingface.co/docs/accelerate/v0.4.0/accelerator.html#accelerate.Accelerator.save

https://github.com/huggingface/accelerate/tree/main